## This notebook imports the budget CSV, turns it into a tree (networkx directed graph), and exports it as a json file to be used by a D3 frontend.

In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd

import sys
sys.path.append('/home/connor/workspace/py_utils')
import graph_helpers as gh

In [2]:
#https://data.richmondgov.com/Well-Managed-Government/City-Budget-General-Fund/7nru-hsrx
df_b = pd.read_csv("City_Budget_-_General_Fund.csv")

In [3]:
df_b_18 = df_b[(df_b.Period == 'FY18')]

In [5]:
G = nx.DiGraph()
G.add_node(gh.setNewID(G),name='RVA_2018_Budget', type='root')

In [6]:
df_accounts = df_b_18[['Agency_Descr','Cost Center_Description','Account_Description', 'Service_Descr', 'Budget']]\
                .groupby(['Agency_Descr','Cost Center_Description', 'Service_Descr','Account_Description'])\
                .sum().reset_index().dropna().rename(columns={'Cost Center_Description': 'cost_center',
                                                             'Agency_Descr': 'agency', 'Account_Description': 'account',
                                                             'Service_Descr': 'service'})
        
df_services = df_accounts.groupby(['agency','cost_center', 'service'])\
                .sum().reset_index().dropna()

df_costCenters = df_services.groupby(['agency','cost_center']).sum().reset_index().dropna()

df_agencies = df_costCenters.groupby(['agency']).sum().reset_index().dropna()

In [7]:
#add agencies and link to root
# root = gh.getNodeByName(G, 'RVA_2018_Budget')
    
for index, row in df_agencies.iterrows():
    name = row.agency
    newID = gh.setNewID(G)
    G.add_node(newID, name=name, type='agency')
    G.add_edge(1, newID)

In [8]:
#add cost centers and link to agency
    
for index, row in df_costCenters.iterrows():
    name = row.cost_center
    newID = gh.setNewID(G)
    G.add_node(newID, name=name, type='cost_center')
    parent = gh.getNodeByName(G, row.agency)
    G.add_edge(parent, newID)

In [9]:
#add services and link to cost center, then add child accounts and link to service
    
for index, row in df_services.iterrows():
    name = row.service
    newID = gh.setNewID(G)
    
    G.add_node(newID,name=name, type='service')
    
    parent = gh.getNodeByName(G, row.cost_center)
    
    G.add_edge(parent, newID)
    
    df_children = df_accounts[(df_accounts.service == name) & (df_accounts.cost_center == row.cost_center)]
    
    for index, ch_row in df_children.iterrows():
        ch_name = ch_row.account
        ch_newID = gh.setNewID(G)
        
        G.add_node(ch_newID,name=ch_name, type='account', budget=ch_row.Budget)
        
        G.add_edge(newID, ch_newID)

In [10]:
nx.write_gpickle(G, "budget_sunbirst_graph.p")

In [13]:
import json

from networkx.readwrite import json_graph

In [14]:
data = json_graph.tree_data(G,root=1)
datastr = json.dumps(data)
data = json.loads(datastr)

In [15]:
with open('local_flask/static/budget_sunbirst.json', 'w') as outfile:
    json.dump(data, outfile)